In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.master("local").appName("GoogleDriveExample").getOrCreate()


In [ ]:
# File path from Google Drive
file_path = '/content/drive/My Drive/1_practice_datasets/ultimate_datasets/trendytech/orders/orders.csv'

# Read CSV file into a Spark DataFrame
spark_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
spark_df.show()


+---+---------------------+-----+---------------+
|  1|2013-07-25 00:00:00.0|11599|         CLOSED|
+---+---------------------+-----+---------------+
|  2|  2013-07-25 00:00:00|  256|PENDING_PAYMENT|
|  3|  2013-07-25 00:00:00|12111|       COMPLETE|
|  4|  2013-07-25 00:00:00| 8827|         CLOSED|
|  5|  2013-07-25 00:00:00|11318|       COMPLETE|
|  6|  2013-07-25 00:00:00| 7130|       COMPLETE|
|  7|  2013-07-25 00:00:00| 4530|       COMPLETE|
|  8|  2013-07-25 00:00:00| 2911|     PROCESSING|
|  9|  2013-07-25 00:00:00| 5657|PENDING_PAYMENT|
| 10|  2013-07-25 00:00:00| 5648|PENDING_PAYMENT|
| 11|  2013-07-25 00:00:00|  918| PAYMENT_REVIEW|
| 12|  2013-07-25 00:00:00| 1837|         CLOSED|
| 13|  2013-07-25 00:00:00| 9149|PENDING_PAYMENT|
| 14|  2013-07-25 00:00:00| 9842|     PROCESSING|
| 15|  2013-07-25 00:00:00| 2568|       COMPLETE|
| 16|  2013-07-25 00:00:00| 7276|PENDING_PAYMENT|
| 17|  2013-07-25 00:00:00| 2667|       COMPLETE|
| 18|  2013-07-25 00:00:00| 1205|         CLOSED|


In [ ]:
import os
# Change the path to the specific folder in your Google Drive
os.listdir('/content/drive/My Drive/1_practice_datasets/ultimate_datasets/trendytech/orders')


['orders.csv', 'orders_1gb.csv', 'ordersparquet']

In [ ]:
import pandas as pd

# Load your dataset
dataset_path = ''
data = pd.read_csv(dataset_path)

# Display the first few rows
data.head()

,1,2013-07-25 00:00:00.0,11599,CLOSED
0,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
1,3,2013-07-25 00:00:00.0,12111,COMPLETE
2,4,2013-07-25 00:00:00.0,8827,CLOSED
3,5,2013-07-25 00:00:00.0,11318,COMPLETE
4,6,2013-07-25 00:00:00.0,7130,COMPLETE


In [ ]:
orders_base=spark.sparkContext.textFile("/content/orders_1gb.csv")

In [ ]:
#customers_base=spark.sparkContext.textFile("/content/orders_1gb.csv")

In [ ]:
orders_mapped=orders_base.map(lambda x:(x.split(',')[2],x.split(',')[3]))

In [ ]:
orders_mapped.take(15)

[('11599', 'CLOSED'),
 ('256', 'PENDING_PAYMENT'),
 ('12111', 'COMPLETE'),
 ('8827', 'CLOSED'),
 ('11318', 'COMPLETE'),
 ('7130', 'COMPLETE'),
 ('4530', 'COMPLETE'),
 ('2911', 'PROCESSING'),
 ('5657', 'PENDING_PAYMENT'),
 ('5648', 'PENDING_PAYMENT'),
 ('918', 'PAYMENT_REVIEW'),
 ('1837', 'CLOSED'),
 ('9149', 'PENDING_PAYMENT'),
 ('9842', 'PROCESSING'),
 ('2568', 'COMPLETE')]

In [ ]:
customers_base=spark.sparkContext.textFile("/content/part-00000")

In [ ]:
customers_base.take(5)

['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725',
 '4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069',
 '5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725']

In [ ]:
customers_mapped=customers_base.map(lambda x:(x.split(",")[0],x.split(",")[8]))

In [ ]:
customers_mapped.take(5)

[('1', '78521'),
 ('2', '80126'),
 ('3', '00725'),
 ('4', '92069'),
 ('5', '00725')]

In [ ]:
joined_rdd=customers_mapped.join(orders_mapped)

In [ ]:
joined_rdd.take(5)

[(('11600', '00725'), 'CLOSED'),
 (('257', '00791'), 'PENDING_PAYMENT'),
 (('12112', '00725'), 'COMPLETE'),
 (('8828', '00725'), 'CLOSED'),
 (('11319', '00612'), 'COMPLETE')]

In [ ]:
joined_rdd.saveAsTextFile("data/orders_join")

In [ ]:
customers_broadcast=spark.sparkContext.broadcast(customers_mapped.collect())

In [ ]:
def get_pincode(customer_id):
  try:
    return   customers_broadcast.value[customer_id]
  except:
    return "-1"

In [ ]:
joined_rdd=orders_mapped.map(lambda x:(get_pincode(int(x[0])),x[1]))

In [ ]:
joined_rdd.saveAsTextFile("data/broadcastresults")

In [ ]:
orders_base=spark.sparkContext.textFile("/content/orders_1gb.csv")

In [ ]:
orders_base.getNumPartitions()

34

In [ ]:
# to increase or decrease the partition

In [ ]:
#"repartition " can increase or decrease the partition
orders_base.repartition(30).getNumPartitions()

30

In [ ]:
orders_filtered=orders_base.filter(lambda x:x.split(",")[3] != 'PENDING_PAYMENT')

In [ ]:
orders_mapped=orders_filtered.map(lambda x:(x.split(",")[2],1))

In [ ]:
orders_mapped.take(5)

[('11599', 1), ('12111', 1), ('8827', 1), ('11318', 1), ('7130', 1)]

In [ ]:
orders_reduce=orders_mapped.reduceByKey(lambda x,y:x+y)

In [ ]:
result =orders_reduce.filter(lambda x: int(x[0])<501)

In [ ]:
result.take(5)

[('58', 2625), ('381', 1500), ('51', 3000), ('226', 1500), ('393', 750)]

**Understanding dataframes**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
orders_df=spark.read\
.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/content/orders_wh.csv")

In [ ]:
orders_df.show()
orders_df.printSchema()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 00:00:00|      11599|         CLOSED|
|       2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
|       6|2013-07-25 00:00:00|       7130|       COMPLETE|
|       7|2013-07-25 00:00:00|       4530|       COMPLETE|
|       8|2013-07-25 00:00:00|       2911|     PROCESSING|
|       9|2013-07-25 00:00:00|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSIN

In [ ]:
 #to change the name of the schema we can apply the transformation on the dataframe
transformed_df1 =orders_df.withColumnRenamed("order_status","status")


In [ ]:
from pyspark.sql.functions import*

In [ ]:
transformed_df2 = transformed_df1.withColumn("order_date_new", to_timestamp("order_date"))


In [ ]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



shortcuts

In [ ]:
spark.read \
.json("/content/orders.json").show()

+-----------+--------------------+--------+---------------+
|customer_id|          order_date|order_id|   order_status|
+-----------+--------------------+--------+---------------+
|      11599|2013-07-25 00:00:...|       1|         CLOSED|
|        256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|      12111|2013-07-25 00:00:...|       3|       COMPLETE|
|       8827|2013-07-25 00:00:...|       4|         CLOSED|
|      11318|2013-07-25 00:00:...|       5|       COMPLETE|
|       7130|2013-07-25 00:00:...|       6|       COMPLETE|
|       4530|2013-07-25 00:00:...|       7|       COMPLETE|
|       2911|2013-07-25 00:00:...|       8|     PROCESSING|
|       5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|       5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
|        918|2013-07-25 00:00:...|      11| PAYMENT_REVIEW|
|       1837|2013-07-25 00:00:...|      12|         CLOSED|
|       9149|2013-07-25 00:00:...|      13|PENDING_PAYMENT|
|       9842|2013-07-25 00:00:...|      

NameError: name 'git' is not defined